# Example to get energies (total, electronic, nuclear, heat of formation, orbital), charges, dipole, forces, density matrix.

In [1]:
import sys
### path to PYSEQM ###
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/")

import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

/home/maxim/anaconda3/envs/hipnn_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Decorating your function! <function KSA_XL_BOMD.one_step at 0x7f435982a670>


In [2]:
%%time

### create molecule object:
species = torch.as_tensor([[8,6,1,1],
                           [8,6,1,1],
                           [8,8,6,0]], # zero-padding for batching
                          dtype=torch.int64, device=device)

coordinates = torch.tensor([
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.23,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [0.0,0.0,0.0]            # zero-padding for batching
                              ]
                            ], device=device)

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecules = Molecule(const, seqm_parameters, coordinates, species).to(device)

### Create electronic structure driver:
esdriver = Electronic_Structure(seqm_parameters).to(device)

### Run esdriver on molecules:
esdriver(molecules)

CPU times: user 3.99 s, sys: 605 ms, total: 4.59 s
Wall time: 4.68 s


In [3]:
print(' Total Energy (eV):\n', molecules.Etot)
print('\n Electronic Energy (eV): ', molecules.Eelec)
print('\n Nuclear Energy (eV):\n', molecules.Enuc)
print('\n Heat of Formation (ev):\n', molecules.Hf)
print('\n HOMO-LUMO gap (eV):\n', molecules.e_gap)
print('\n Orbital energies (eV):\n', molecules.e_mo)
print('\n Charges:\n', molecules.q)
print('\n Dipole:\n', molecules.d)
print('\n Forces (eV/A):\n', molecules.force)

 Total Energy (eV):
 tensor([-475.5819, -475.5819, -759.2461], device='cuda:0')

 Electronic Energy (eV):  tensor([ -864.9940,  -864.9940, -1504.4581], device='cuda:0')

 Nuclear Energy (eV):
 tensor([389.4122, 389.4122, 745.2120], device='cuda:0')

 Heat of Formation (ev):
 tensor([-1.3620, -1.3620,  6.3444], device='cuda:0')

 HOMO-LUMO gap (eV):
 tensor([11.5918, 11.5918,  9.3511], device='cuda:0')

 Orbital energies (eV):
 tensor([[-39.2305, -25.0153, -17.1715, -16.2761, -14.6381, -10.7554,   0.8365,
           3.1509,   3.9780,   6.0957,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000],
        [-39.2305, -25.0153, -17.1715, -16.2761, -14.6381, -10.7554,   0.8365,
           3.1509,   3.9780,   6.0957,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000],
        [-46.6531, -35.1909, -24.5378, -20.3546, -20.2776, -15.5339, -11.1724,
         -11.1534,  -1.8023,  -0.0903,   2.2116,   6.4507,   0.0000,   0.0000,
           0.0000,   0.0000]], devi

In [4]:
print('\n Density matrix:\n', molecules.dm)


 Density matrix:
 tensor([[[ 1.9076e+00, -2.7516e-01,  0.0000e+00,  0.0000e+00,  1.3505e-01,
          -2.8679e-01,  0.0000e+00,  0.0000e+00,  6.1675e-03,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  6.1675e-03,  0.0000e+00,  0.0000e+00,
           0.0000e+00],
         [-2.7516e-01,  1.1585e+00,  0.0000e+00,  0.0000e+00,  5.0981e-01,
          -7.8702e-01,  0.0000e+00,  0.0000e+00, -9.9677e-02,  0.0000e+00,
           0.0000e+00,  0.0000e+00, -9.9677e-02,  0.0000e+00,  0.0000e+00,
           0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  1.9076e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  3.0857e-01,  0.0000e+00, -2.0125e-01,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  2.0125e-01,  0.0000e+00,  0.0000e+00,
           0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.2997e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  9.5404e-01,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         

In [5]:
molecules.coordinates

tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 1.2200,  0.0000,  0.0000],
         [ 1.8200,  0.9400,  0.0000],
         [ 1.8200, -0.9400,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 1.2200,  0.0000,  0.0000],
         [ 1.8200,  0.9400,  0.0000],
         [ 1.8200, -0.9400,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 1.2300,  0.0000,  0.0000],
         [ 1.8200,  0.9400,  0.0000],
         [ 0.0000,  0.0000,  0.0000]]], device='cuda:0', requires_grad=True)